In [52]:
import pandas as pd
import sys
from sklearn.pipeline import Pipeline
import numpy as np


In [53]:
sys.path.append("C:\\Users\\Bina4\\Desktop\\Guy_hafifa\\scrabble")

In [54]:
games = pd.read_csv('../data/games.csv', index_col='game_id')
train = pd.read_csv('../data/train.csv', index_col='game_id')
turns = pd.read_csv('../data/turns.csv', index_col='game_id')

In [55]:
import functions
import transformers
import importlib

importlib.reload(functions)
importlib.reload(transformers)
from transformers.columns_setter_transformer import ColumnsSetterTransformer
from transformers.extract_set_column_transformer import ExtractSetColumnsTransformer
from transformers.name_dropper_transformer import NameDropperTransformer
from transformers.select_transformer import SelectTransformer
from transformers.select_rows_transformer import SelectRowsTransformer
from transformers.series_from_group_transformer import SeriesFromGroupTransformer
from transformers.map_set_transformer import MapSetTransformer




from functions.bot_extractor import BotExtarctor
from functions.is_bot_extractor import IsBotExtarctor
importlib.reload(functions.is_bot_extractor)
importlib.reload(functions.bot_extractor)
importlib.reload(transformers.columns_setter_transformer)
importlib.reload(transformers.extract_set_column_transformer)
importlib.reload(transformers.name_dropper_transformer)
importlib.reload(transformers.select_transformer)
importlib.reload(transformers.select_rows_transformer)
importlib.reload(transformers.series_from_group_transformer)
importlib.reload(transformers.map_set_transformer)





<module 'transformers.map_set_transformer' from 'C:\\Users\\Bina4\\Desktop\\Guy_hafifa\\scrabble\\transformers\\map_set_transformer.py'>

In [56]:
names = ['BetterBot', 'STEEBot', 'HastyBot']
bots = BotExtarctor(names)(train)
is_player = ~IsBotExtarctor(names)(train)


In [57]:
test = train
transformer = SelectTransformer(is_player)
transformer.transform(test)


,nickname,score,rating
game_id,,,
1,stevy,429,1500
3,davidavid,440,1811
4,Inandoutworker,119,1473
5,stevy,325,1500
6,HivinD,378,2029
...,...,...,...
72767,HAYDEN,340,1395
72770,samsiah06,97,1332
72771,BB-8,390,1500


In [58]:
train_pipe = (Pipeline([('bot_names', ColumnsSetterTransformer({'bot_name': BotExtarctor(names),})),
                   ('players_selection', SelectTransformer(is_player)),
                   ('set_bot_training', ColumnsSetterTransformer({'bot_rating': train[~is_player]['rating']})),
                   ('name_dropper', NameDropperTransformer(['nickname']))
                   ]))

games_pipe = (Pipeline([('extract_first', ColumnsSetterTransformer({'is_bot_first': IsBotExtarctor(names, name_col='first')})),
                        ('drop_first', NameDropperTransformer('first'))
                       ]))

new_train = train_pipe.transform(train)
new_games = games_pipe.transform(games).merge(new_train, left_index=True, right_index=True)
new_games

,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,is_bot_first,score,rating,bot_name,bot_rating
game_id,,,,,,,,,,,,,,,
1,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274,True,429,1500,BetterBot,1637
3,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262,True,440,1811,BetterBot,2071
4,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,350.861141,True,119,1473,BetterBot,1936
5,regular,STANDARD,0,2022-09-06 04:31:36,NWL20,1200,0,CASUAL,1,642.688722,True,325,1500,STEEBot,1844
6,regular,STANDARD,0,2022-08-21 14:56:35,CSW21,900,0,RATED,1,426.950541,False,378,2029,STEEBot,2143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72767,regular,TIME,0,2022-09-05 04:35:56,CSW21,3600,0,CASUAL,1,3669.988413,True,340,1395,BetterBot,1952
72770,regular,TIME,0,2022-09-04 00:13:24,CSW21,3600,0,RATED,1,3662.659287,True,97,1332,HastyBot,2237
72771,regular,STANDARD,0,2022-09-19 03:28:44,ECWL,900,0,CASUAL,10,236.133634,True,390,1500,HastyBot,1614


In [59]:
# new_games = games.merge(new_train, left_index=True, right_index=True)
# new_games

In [60]:
# is_player = ~IsBotExtarctor(names)(train)
# bots = BotExtarctor(names)(train)
# train_no_bots = train[is_player]
# train_no_bots['bot_name'] = bots
# train_no_bots['bot_rating'] = train[~is_player]['rating']
# train_no_bots = train_no_bots.drop(columns=['nickname'])
# games_with_rating = games.merge(train_no_bots, left_index=True, right_index=True)
# train_no_bots


In [61]:
turns

,turn_number,nickname,rack,location,move,points,score,turn_type
game_id,,,,,,,,
1,1,BetterBot,DDEGITT,8G,DIG,10,10,Play
1,2,stevy,AEHOPUX,7H,HAP,18,18,Play
1,3,BetterBot,DEELTTU,6I,LUTE,16,26,Play
1,4,stevy,EMORSUX,5K,UM,16,34,Play
1,5,BetterBot,ACDEITU,L5,..DICATE,28,54,Play
...,...,...,...,...,...,...,...,...
72773,22,adola,ABINRRU,15N,IN,18,376,Play
72773,23,HastyBot,EGHIIP,H12,.HIG,24,331,Play
72773,24,adola,ABRRU,15E,BRA.,7,383,Play


In [62]:
trans = SelectRowsTransformer(train.index)
train_turns = trans.transform(turns)

In [63]:
turns_pipe = Pipeline([('is_player_extractor', ColumnsSetterTransformer({'is_player_turn': IsBotExtarctor(names, neg=True)})),
                       ('word_info_mappers', MapSetTransformer({'move_len': (lambda x: len(x) if type(x) == str else np.NAN, 'move'),
                                                                'reused_num': (lambda x: x.count('.') if type(x) == str else np.NAN, 'move'),
                                                                'jokers_num': (lambda x: sum(1 for c in x if c.islower()) if (type(x) == str) and (x not in ['(challenge)', '(time)']) else np.NAN, 'move')
                                                                })),
                       ('to_series', SeriesFromGroupTransformer('game_id', 'turn_number'))
                       ])

new_turns = turns_pipe.transform(turns)
new_turns

1                      nickname     rack location      ...
2                      nickname     rack location      ...
3                      nickname     rack location      ...
4                           nickname     rack location ...
5                    nickname     rack location       m...
                               ...                        
72769                 nickname     rack location      m...
72770                  nickname     rack location      ...
72771                 nickname     rack location     mo...
72772                  nickname     rack location      ...
72773                 nickname     rack location     mo...
Length: 72773, dtype: object

In [64]:
# new_turns.groupby('game_id').sum()['jokers_num']

In [65]:
new_turns[new_turns['jokers_num'] == 4]

KeyError: 'jokers_num'

In [66]:
new_turns.loc[6817]

,nickname,rack,location,move,points,score,turn_type,is_player_turn,move_len,reused_num,jokers_num
turn_number,,,,,,,,,,,
1,BetterBot,CEENOOO,NaN,-EOO,0,0,Exchange,False,4.0,0.0,0.0
2,Scrabbeduto,?AEEGTW,8G,WAGE,16,16,Play,True,4.0,0.0,0.0
3,BetterBot,CEGLNOR,K3,CONGER,27,27,Play,False,6.0,0.0,0.0
4,Scrabbeduto,?EETUXY,L4,XU,38,54,Play,True,2.0,0.0,0.0
5,BetterBot,HKLPRVV,NaN,-KLPVV,0,27,Exchange,False,6.0,0.0,0.0
6,Scrabbeduto,?EEITTY,G8,.YE,13,67,Play,True,3.0,1.0,0.0
7,BetterBot,?HIOORS,M5,bOORISH,63,90,Play,False,7.0,0.0,1.0
8,Scrabbeduto,?AAEITT,8M,.AT,9,76,Play,True,3.0,1.0,0.0
9,BetterBot,AIOPSUV,L11,OVA,17,107,Play,False,3.0,0.0,0.0


In [67]:
more_games_pipe = Pipeline([('assign_turns', ColumnsSetterTransformer({'turns': new_turns}))])

new_games = more_games_pipe.transform(new_games)
new_games.head()

,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,is_bot_first,score,rating,bot_name,bot_rating,turns
game_id,,,,,,,,,,,,,,,,
1,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274,True,429,1500,BetterBot,1637,nickname rack location ...
3,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262,True,440,1811,BetterBot,2071,nickname rack location ...
4,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,350.861141,True,119,1473,BetterBot,1936,nickname rack location ...
5,regular,STANDARD,0,2022-09-06 04:31:36,NWL20,1200,0,CASUAL,1,642.688722,True,325,1500,STEEBot,1844,nickname rack location m...
6,regular,STANDARD,0,2022-08-21 14:56:35,CSW21,900,0,RATED,1,426.950541,False,378,2029,STEEBot,2143,nickname rack location mo...


In [73]:
new_games['turns'][4]

,nickname,rack,location,move,points,score,turn_type,is_player_turn,move_len,reused_num,jokers_num
turn_number,,,,,,,,,,,
1,BetterBot,ADEMSTX,8D,MAXED,36,36,Play,False,5.0,0.0,0.0
2,Inandoutworker,DISTVWY,9E,WIT,24,24,Play,True,3.0,0.0,0.0
3,BetterBot,DEJRSTU,7G,JET,40,76,Play,False,3.0,0.0,0.0
4,Inandoutworker,ADLOSVY,J7,SLAY,26,50,Play,True,4.0,0.0,0.0
5,BetterBot,DERRRSU,E4,REDR..,20,96,Play,False,6.0,2.0,0.0
6,Inandoutworker,BBDGOUV,NaN,-BBGOUV,0,50,Exchange,True,7.0,0.0,0.0
7,BetterBot,EFIMRSU,D1,FUME,30,126,Play,False,4.0,0.0,0.0
8,Inandoutworker,DEEEGOR,1A,REE.,21,71,Play,True,4.0,1.0,0.0
9,BetterBot,EINNORS,E4,......N,11,137,Play,False,7.0,6.0,0.0


In [68]:
moments_pipe = Pipeline([('points_moments', MapSetTransformer({'p_points_1_moment': (lambda turns: (turns[turns['is_player_turn']]['points']).mean(), 'turns'),
                                                            'p_points_std': (lambda turns: turns[turns['is_player_turn']]['points'].std(), 'turns'),
                                                            'b_points_1_moment': (lambda turns: (turns[~turns['is_player_turn']]['points']).mean(), 'turns'),
                                                            'b_points_std': (lambda turns: turns[~turns['is_player_turn']]['points'].std(), 'turns'),
                                                            'a_points_1_moment': (lambda turns: (turns['points']).mean(), 'turns'),
                                                            'a_points_std': (lambda turns: turns['points'].std(), 'turns'),
                                                            },
))])
res = moments_pipe.transform(new_games)


              

In [69]:
res[['p_points_1_moment', 'p_points_std', 
     'b_points_1_moment', 'b_points_std',
     'a_points_1_moment', 'a_points_std', 'rating']].corr()['rating']

p_points_1_moment    0.399423
p_points_std         0.237703
b_points_1_moment    0.265389
b_points_std         0.069554
a_points_1_moment    0.413806
a_points_std         0.191779
rating               1.000000
Name: rating, dtype: float64

In [79]:
word_pipe = Pipeline([('worder', MapSetTransformer({'p_len_avg': (lambda turns: (turns[turns['is_player_turn']]['move_len']).mean(), 'turns'),
                                                    'p_len_std': (lambda turns: (turns[turns['is_player_turn']]['move_len']).std(), 'turns'),
                                                    'p_len_max': (lambda turns: (turns[turns['is_player_turn']]['move_len']).max(), 'turns'),
                                                    'p_len_sum': (lambda turns: (turns[turns['is_player_turn']]['move_len']).sum(), 'turns'),
                                                    'p_len_median': (lambda turns: (turns[turns['is_player_turn']]['move_len']).median(), 'turns'),
                                                    'p_turns_num': (lambda turns: (turns[turns['is_player_turn']]).shape[0], 'turns'),
                                                    'p_points_median': (lambda turns: (turns[turns['is_player_turn']])['points'].median(), 'turns'),
                                                    'b_points_median': (lambda turns: (turns[~turns['is_player_turn']])['points'].median(), 'turns'),
                                                    'a_points_median': (lambda turns: (turns)['points'].median(), 'turns'),


                                                  })
                     )])
res = word_pipe.transform(new_games)
res.head()

,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,...,turns,p_len_avg,p_len_std,p_len_max,p_len_sum,p_len_median,p_turns_num,p_points_median,b_points_median,a_points_median
game_id,,,,,,,,,,,,,,,,,,,,,
1,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274,...,nickname rack location ...,3.857143,1.703261,7.0,54.0,3.0,14,25.0,24.0,24.0
3,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262,...,nickname rack location ...,4.357143,1.691933,7.0,61.0,4.0,14,28.0,22.0,25.0
4,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,350.861141,...,nickname rack location ...,2.928571,1.384768,7.0,41.0,2.5,14,5.5,24.0,16.0
5,regular,STANDARD,0,2022-09-06 04:31:36,NWL20,1200,0,CASUAL,1,642.688722,...,nickname rack location m...,3.375000,1.147461,6.0,54.0,3.0,16,20.5,21.0,21.0
6,regular,STANDARD,0,2022-08-21 14:56:35,CSW21,900,0,RATED,1,426.950541,...,nickname rack location mo...,4.750000,0.753778,6.0,57.0,5.0,12,28.5,26.0,28.0


In [80]:
res[['p_len_avg', 'p_len_std', 'p_len_max', 'p_len_sum', 'p_len_median',
     'p_turns_num', 'p_points_median', 'b_points_median', 'a_points_median',
       'rating']].corr()['rating']

p_len_avg          0.234471
p_len_std          0.226608
p_len_max          0.173144
p_len_sum         -0.061038
p_len_median       0.142012
p_turns_num       -0.254070
p_points_median    0.369247
b_points_median    0.270809
a_points_median    0.405427
rating             1.000000
Name: rating, dtype: float64

In [ ]:
def count(phrase):
    def f(str):
        return str.lower().count(phrase)
    return f


char_pipe = Pipeline([('worder', MapSetTransformer({'p_len_avg': (lambda turns: (turns[turns['is_player_turn']]['move']).map(count('')).sum(), 'turns'),



                                                  })
                     )])
res = word_pipe.transform(new_games)
res.head()

In [ ]:
turns[turns['move'].isna()]

,turn_number,nickname,rack,location,move,points,score,turn_type
game_id,,,,,,,,
377,27,NurdiniDelisha,ADFNTTU,O8,NaN,16,249,NaN
409,22,akiff_faleel,AANU,14J,NaN,12,350,NaN
420,23,Rick,AGINU,M12,NaN,19,409,NaN
536,13,HastyBot,AAEILNN,F5,NaN,14,316,NaN
630,2,220674_ZAVEPHUA,AAEINNR,7H,NaN,11,11,NaN
...,...,...,...,...,...,...,...,...
72252,10,akiff_faleel,CGLLNUX,D10,NaN,12,238,Play
72395,18,gppa,AEINRSU,A14,NaN,35,272,NaN
72467,7,Saryk,?AALNOU,9F,NaN,11,74,NaN


In [ ]:
new_games['turns'][1]

,nickname,rack,location,move,points,score,turn_type,is_player_turn,move_len,reused_num,jokers_num
turn_number,,,,,,,,,,,
1,BetterBot,DDEGITT,8G,DIG,10,10,Play,False,3.0,0.0,0.0
2,stevy,AEHOPUX,7H,HAP,18,18,Play,True,3.0,0.0,0.0
3,BetterBot,DEELTTU,6I,LUTE,16,26,Play,False,4.0,0.0,0.0
4,stevy,EMORSUX,5K,UM,16,34,Play,True,2.0,0.0,0.0
5,BetterBot,ACDEITU,L5,..DICATE,28,54,Play,False,8.0,2.0,0.0
6,stevy,EEOORSX,M11,OXO,31,65,Play,True,3.0,0.0,0.0
7,BetterBot,AAEJRSU,5E,RAJAS,29,83,Play,False,5.0,0.0,0.0
8,stevy,AEEFIRS,9B,FAERIES,72,137,Play,True,7.0,0.0,0.0
9,BetterBot,ELNNORU,C3,NEURON.L,68,151,Play,False,8.0,1.0,0.0
